# ElMO + Corrección de errores

- Agrandemos la longitud de la secuencia
- Veamos de manejar los hashtags de alguna otra manera



In [1]:
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.sequence import pad_sequences


max_length = 45

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=False)

def tokenize(tweet):
    tokens = tokenizer.tokenize(tweet)
    
    ret = []
    for token in tokens:
        if token[0] == "#":
            ret.append(token[1:])
        elif token[0] == "@":
            ret.append("@user")
        else:
            ret.append(token)
    
    return ret
    

def preprocess_tweet(tweet):
    tokens = tokenize(tweet)
    
    if len(tokens) >= max_length:
        tokens = tokens[:max_length]
    else:
        tokens = tokens + [''] * (max_length - len(tokens))
    return tokens


text_train = [preprocess_tweet(tweet) for tweet in df_train["text"].values]
text_dev = [preprocess_tweet(tweet) for tweet in df_dev["text"].values]

Using TensorFlow backend.


In [3]:
%%capture
from elmoformanylangs import Embedder

e = Embedder("../../../models/elmo/es/")

Carguemos embeddings

In [4]:
import numpy as np

print(text_train[0])


X_train = np.array(e.sents2elmo(text_train))
X_dev = np.array(e.sents2elmo(text_dev))

['easyjet', 'quiere', 'duplicar', 'el', 'número', 'de', 'mujeres', 'piloto', "'", 'verás', 'tú', 'para', 'aparcar', 'el', 'avión', '..', 'http://t.co/46NuLkm09x', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [5]:
X_train.shape, X_dev.shape, y_train.shape, y_dev.shape

((4500, 45, 1024), (500, 45, 1024), (4500,), (500,))

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, Dropout, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam

optimizer_args = {
    "lr": 0.0005,
    "decay": 0.01
}

embedding_dim = 1024

model = Sequential()
model.add(Bidirectional(LSTM(256, input_shape=(max_length, embedding_dim))))
model.add(Dropout(0.80))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.55))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit(X_train, y_train.values, validation_data=(X_dev, y_dev), epochs=25, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/25
4500/4500 [==============================] - 20s 4ms/step - loss: 0.7479 - acc: 0.5467 - val_loss: 0.6671 - val_acc: 0.6260
Epoch 2/25
4500/4500 [==============================] - 18s 4ms/step - loss: 0.6770 - acc: 0.6036 - val_loss: 0.6530 - val_acc: 0.6480
Epoch 3/25
4500/4500 [==============================] - 18s 4ms/step - loss: 0.6504 - acc: 0.6278 - val_loss: 0.6289 - val_acc: 0.6820
Epoch 4/25
4500/4500 [==============================] - 18s 4ms/step - loss: 0.6115 - acc: 0.6656 - val_loss: 0.6017 - val_acc: 0.7000
Epoch 5/25
4500/4500 [==============================] - 18s 4ms/step - loss: 0.5826 - acc: 0.7053 - val_loss: 0.5786 - val_acc: 0.6920
Epoch 6/25
4500/4500 [==============================] - 18s 4ms/step - loss: 0.5696 - acc: 0.7147 - val_loss: 0.5562 - val_acc: 0.7220
Epoch 7/25
4500/4500 [==============================] - 18s 4ms/step - loss: 0.5435 - acc: 0.7298 - val_loss: 0.5401 - val_acc: 0.7400
Epoch 8/

In [7]:
model.fit(X_train, y_train.values, validation_data=(X_dev, y_dev), epochs=2, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/2
4500/4500 [==============================] - 18s 4ms/step - loss: 0.3593 - acc: 0.8520 - val_loss: 0.4341 - val_acc: 0.8100
Epoch 2/2
4500/4500 [==============================] - 18s 4ms/step - loss: 0.3454 - acc: 0.8564 - val_loss: 0.4360 - val_acc: 0.8100


In [8]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 1s 2ms/step
Loss        : 0.4360
Accuracy    : 0.8100
Precision   : 0.7773
Recall      : 0.8018
F1          : 0.7894


## Error Analysis

Vamos a ver los tweets con mayores errores

In [9]:
df_dev["proba"] = model.predict_proba(X_dev)


true_positives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] >= 0.5)].copy()
true_negatives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] < 0.5)].copy()

false_positives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] > 0.5)].copy()
false_positives.sort_values("proba", ascending=False, inplace=True)


false_negatives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] < 0.5)].copy()
false_negatives.sort_values("proba", ascending=True, inplace=True)

conf_matrix = pd.DataFrame([
    {"real":"hs=1", "pred_true": len(true_positives), "pred_false": len(false_negatives)},
    {"real":"hs=0", "pred_true": len(false_positives), "pred_false": len(true_negatives)}
])



conf_matrix.set_index("real", inplace=True)

print("Falsos negativos: {}".format(len(false_negatives)))
print("Falsos positivos: {}".format(len(false_positives)))

conf_matrix[["pred_true", "pred_false"]]

Falsos negativos: 44
Falsos positivos: 51


,pred_true,pred_false
real,,
hs=1,178,44
hs=0,51,227


## Falsos Negativos

Veamos los 20 falsos negativos en los cuales nuestro modelo se equivoca más

In [61]:
pd.set_option('max_colwidth', 300)

false_negatives[["text", "proba"]]


,text,proba
id,,
21727,"España crea su primer 'campo de refugiados' en la bahía de Algeciras https://t.co/l7Zmh2kvPN vía @elmundoes Gran noticia para el contribuyente español. Españoles a pagar, a pagar. Las pensiones pensad en las pensiones. Esto la sanidad como es gratis podemos dejar de pagarla no?",0.023318
21535,#VOX critica duramente a ⁦@pablocasado_⁩ “Les da la bienvenida como si fuera un cartel del welcome de #Carmena “ ¡Expulsión inmediata de moromierdas! https://t.co/HaySDibj2l,0.031317
21072,"Voy andando por la calle y me han dicho cosas en árabe ya que me estas acosando dime lo en español para que te entienda, asique como yo no les entiendo me he cagado en su puta madre en árabe para que ellos a mi si.",0.043970
22008,"@Namurii @medicaster_xz @Yorha2d @KeloKlau_ Se merecía de puta y no quiere que le metan la verga, jajaja no mamen",0.046136
22898,"""@monicaoltra : """"Sólo seis CCAA estamos absorbiendo la inmigración, no damos abasto""""......... A VER SI POR FIN OS DAIS CUENTA QUE LA SOLIDARIDAD Y EL BUENISMO TIENE LÍMITES. DEJAD LA DEMAGOGIA DE UNA VEZ. NO PUEDE HABER INMIGRACIÓN ILEGAL ILIMITADA. https://t.co/gwCwCCv9wb""",0.066957
21305,"""Tú explicas el patriarcado como """"simios negratas"""" no vengas a exigir nada a las explicaciones de otras cuando se ha explicado desde hace años https://t.co/7VzZRnjNu1""",0.081964
24822,@Relatofeminista 'A mí sólo me gustan las tías si son guarras'. Para mí está claro que si usa guarra para adjetivar el tipo de mujeres que le gusta no debe de asociarle las mismas connotaciones negativas que presuponéis. Tengo amigas que se autoadjetivan,0.084501
24488,Un indocumentado...habla y habla y no para y acaba hablando su ignorancia.Empezamos bien. https://t.co/SMRsqjtEBT,0.087576
24481,En que se parece superman a una mujer lista: En que los dos son personajes de ciencia-ficción.,0.091507


¿Cómo los tokenizamos?

In [20]:
df_dev.loc[21841]["text"]

'La otra no noticia del día es la visita del sonriente Casado a Algeciras, a visitar en Ceuta y Algeciras, no a los guardias civiles heridos tras la violenta entrada de ilegales del otro día, sino a un grupo de subsaharianos que han entrado en nuestro país de forma igualmente ileg'

## Elmo y OOV words

¿Qué hace elmo con palabras extrañas como "negratas", "subsaharianos", etc?

In [59]:
vocabulary = e.word_lexicon.keys()

"negratas" in vocabulary

False

In [60]:
"subsahariano" in vocabulary

False

Ok. Estamos perdiendo información con esto

In [51]:
vecs = e.sents2elmo([["negratas"], ["subsahariano"]], 2)

In [52]:
vecs[0].shape, vecs[1].shape

((1, 1024), (1, 1024))

In [57]:
v1 = vecs[0].reshape(-1)
v2 = vecs[1].reshape(-1)

sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))

sim

0.7698738

## Falsos negativos

In [63]:
pd.set_option('max_colwidth', 200)
false_positives[["text", "proba"]]

,text,proba
id,,
24159,"@LeonDeJapon Jajajajajaja tu eres uno, diego jajajajajaja mentira perra perra perrraaaaaaaaaa de mierda jajaja ♡",0.958672
22517,"-¿eres poblana? -soy colombiana -¿y por qué eres güera? en Colombia TODAS son morenas, fui a Colombia 3 veces. -ok, mentí 🙄 solo me pinto el cabello",0.955924
21966,"escúchame ya me jodería jajajajajajajajajajajajajaja si pa ti enseñar es ser una guarra yo soy biennnn guarra, menos mal que no me respeto que si no... https://t.co/yt5fHp99b7",0.946862
21050,@GvyMC @Wismichu Callate puta rata😂😂 ya estas llorando??,0.944250
22416,@Pepe_ElEzequiel Puta!? Puta con las que vos te metes Esa mujer es un ángel enviado por Jesucristo nuestro señor para darnos esa paz que buscamos sólo con verla,0.941651
24402,zorra eres tu que no te sientes hombre y no entiendes que toda mujer es bella,0.930660
22457,"De Colombia tocará emigrar en pateras, como hacen los africanos subsaharianos.",0.920149
24529,@desoir2525 Cállate rata MUDera.eres escoria.,0.908138
21892,@saancisco Callate puta tu eres el traidor.,0.878067


## Usando las traducciones?

Probemos, ya que estamos...

In [71]:
train_synth_en = pd.read_table("../../../data/es/train_es.synth.en.tsv", index_col="id", quoting=csv.QUOTE_NONE)
train_synth_fr = pd.read_table("../../../data/es/train_es.synth.fr.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train_synth_en, y_train_synth_en = train_synth_en["text"], train_synth_en["HS"]
text_train_synth_fr, y_train_synth_fr = train_synth_fr["text"], train_synth_fr["HS"]


X_train_synth_en = e.sents2elmo([preprocess_tweet(tweet) for tweet in text_train_synth_en.values])
X_train_synth_fr = e.sents2elmo([preprocess_tweet(tweet) for tweet in text_train_synth_fr.values])

X_tr = np.vstack((X_train, X_train_synth_en, X_train_synth_fr))
y_tr = np.vstack((y_train.values.reshape(-1, 1), 
                  y_train_synth_en.values.reshape(-1, 1),
                  y_train_synth_fr.values.reshape(-1, 1)
))

In [73]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, Dropout, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam

optimizer_args = {
    "lr": 0.0005,
    "decay": 0.01
}

embedding_dim = 1024

model = Sequential()
model.add(Bidirectional(LSTM(256, input_shape=(max_length, embedding_dim))))
model.add(Dropout(0.70))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit(X_tr, y_tr, validation_data=(X_dev, y_dev), epochs=25, batch_size=32)


Train on 7168 samples, validate on 500 samples
Epoch 1/25
7168/7168 [==============================] - 29s 4ms/step - loss: 0.6832 - acc: 0.6006 - val_loss: 0.6311 - val_acc: 0.6600
Epoch 2/25
7168/7168 [==============================] - 26s 4ms/step - loss: 0.5984 - acc: 0.6874 - val_loss: 0.5714 - val_acc: 0.7240
Epoch 3/25
7168/7168 [==============================] - 26s 4ms/step - loss: 0.5487 - acc: 0.7252 - val_loss: 0.5278 - val_acc: 0.7480
Epoch 4/25
7168/7168 [==============================] - 27s 4ms/step - loss: 0.5056 - acc: 0.7582 - val_loss: 0.5027 - val_acc: 0.7640
Epoch 5/25
7168/7168 [==============================] - 27s 4ms/step - loss: 0.4747 - acc: 0.7780 - val_loss: 0.4912 - val_acc: 0.7780
Epoch 6/25
7168/7168 [==============================] - 28s 4ms/step - loss: 0.4547 - acc: 0.7914 - val_loss: 0.5095 - val_acc: 0.7500
Epoch 7/25
7168/7168 [==============================] - 29s 4ms/step - loss: 0.4378 - acc: 0.8051 - val_loss: 0.4722 - val_acc: 0.7860
Epoch 8/

# Conclusiones

Sirvió mejorar el preprocesamiento. Quedamos en pérdida ~0.43

No estarían convenciendo las traducciones...

Diría que intentemos un ensemble con caracteres o n-gramas.